# Ice shelves

A first demo with icepack.

In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from mpl_toolkits import mplot3d
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import firedrake
from firedrake import Constant, exp, inner, grad, dx
import irksome
from irksome import Dt

## Setup

Here we want to use our own custom mesh, which is stored in a file `ice-shelf.msh`.
Ask me about mesh generation some time...

In [ ]:
mesh = ...

This mesh has two different boundary segments, unlike the one we used before.

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(
    mesh,
    axes=ax,
    boundary_kw={"colors": ["tab:blue", "tab:orange"]},
)
ax.legend(loc="upper right");

Here we're going to create a finite element and a function space.
But we'll instead be using a discontinuous element because the mass conservation equation is hyperbolic.

In [ ]:
thickness_element = ...
Q = ...

Here we also need a function space for the velocity field.

In [ ]:
velocity_element = ...
V = ...

Now we're going to create some algebraic expressions for the initial thickness and velocity.
I came up with these using a *lot* of trial and error.
You can ignore all the messy algebra.

In [ ]:
x = firedrake.SpatialCoordinate(mesh)

inlet_angles = π * np.array([-3/4, -1/2, -1/3, -1/6])
inlet_widths = π * np.array([1/8, 1/12, 1/24, 1/12])

θs = [Constant(θ) for θ in inlet_angles]
δθs = [Constant(δθ) for δθ in inlet_widths]

R = Constant(200e3)

h_in = Constant(350)
H = Constant(100)
δh = Constant(400)

u_in = Constant(300)
δu = Constant(250.0)

h_exprs = []
u_exprs = []
for θ, δθ in zip(θs, δθs):
    v = Constant((firedrake.cos(θ), firedrake.sin(θ)))
    x_0 = R * v
    L = -inner(x - x_0, v)
    W = x - x_0 + L * v
    R_n = 2 * δθ / π * R

    q = firedrake.max_value(0, 1 - (W / R_n)**2)
    h_expr = H + q * ((h_in - H) - δh * L / R)
    h_exprs.append(h_expr)

    u_expr = -exp(-4 * (W / R)**2) * (u_in + δu * L / R) * v
    u_exprs.append(u_expr)

In [ ]:
h_expr = Constant(H)
for expr in h_exprs:
    h_expr = firedrake.max_value(h_expr, expr)

Since the thickness field lives in a space of discontinuous functions, we can't interpolate it like we normally would if we were using continuous basis functions.
Instead, we compue its projection.

In [ ]:
h_0 = ...

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.axis("off")
colors = firedrake.tripcolor(h_0, shading="gouraud", axes=ax)
fig.colorbar(colors, orientation="horizontal");

We can still interpolate the velocity.

In [ ]:
u_expr = sum(u_exprs)
u_0 = firedrake.Function(V).interpolate(u_expr)

## Solve for the velocity

We first need to pick the parameters that we'll use in the momentum balance equation, like the ice rheology, fluidity, etc.

This is the fluidity of ice at about -18${}^\circ$C.
It's in units of MPa${}^{-3}$ years${}^{-1}$.
That's kind of hard to picture.
It's easier to ask: what strain rate would result at a stress of 100 kPa.

In [ ]:
A = Constant(6.5)   # Fluidity in MPa⁻³ yr⁻¹

n = Constant(3)
τ_c = Constant(0.1)
ε_c = ...
print(f"Strain rate at 100 kPa: {1000 * float(ε_c):0.4f} (m / yr) / km")

We're almost ready to try and solve the momentum balance equation.
We have to once again create a function space for the membrane stress.
But this is a symmetric tensor.

In [ ]:
stress_element = ...
Σ = ...

Here we'll have to do something new.
We want to simultaneously solve for the velocity and stress.
So we need to create a new function space which is the product of the velocity and stress spaces.

In [ ]:
Z = ...

We'll initialize our solution variable with the velocity we made up above.
We can access the components of it with the `.sub` method.

In [ ]:
z = firedrake.Function(Z)
z.sub(0).assign(u_0);

In order to create the symbolic form of the problem, we need to get some variables describing the velocity and stress and the corresponding test functions.

In [ ]:
u, M = ...
v, N = ...

Now we'll call out to icepack to make the variational form of the problem.

In [ ]:
from icepack2.model.variational import flow_law, ice_shelf_momentum_balance

F = ...

We need special handling of boundary conditions again because we're working with a mixed function space.

In [ ]:
inflow_ids = [1]
bc = firedrake.DirichletBC(Z.sub(0), u_0, inflow_ids)

Here we're adding a few extra options:
1. specify what quadrature degree we want
2. print out diagnostic information so we can see the solver converge
3. use a different line search method

In [ ]:
pparams = {"form_compiler_parameters": {"quadrature_degree": 6}}
problem = firedrake.NonlinearVariationalProblem(F, z, bc, **pparams)

In [ ]:
sparams = {
    "solver_parameters": {
        "snes_monitor": None,
        "snes_linesearch_type": "nleqerr",
    },
}
solver = firedrake.NonlinearVariationalSolver(problem, **sparams)

Solve the momentum balance equation, I think...?

In [ ]:
z.sub(0).assign(u_0)
z.sub(1).assign(0.0)

solver.solve()

## The simulation

Now that we have an initial velocity and stress, we can start solving the coupled mass and momentum balance equations.
We'll have to make an even bigger function space.

In [ ]:
W = ...

Initialize our solution in the bigger space with the velocity and stress that we just computed and the initial thickness.

In [ ]:
w = firedrake.Function(W)
w.sub(0).assign(z.sub(0))
w.sub(1).assign(z.sub(1))
w.sub(2).assign(h_0);

Extract the components of the solution and test functions.

In [ ]:
u, M, h = ...
v, N, ϕ = ...

Here we'll assume no net accumulation / ablation, i.e. that the rate of snow mass accumulation is equal to the rate of ocean melting.

In [ ]:
a = Constant(0.0)

Creating the dynamics looks similar to before, but now we add on the mass balance.

In [ ]:
from icepack2.model import mass_balance

F = ...

In [ ]:
t = Constant(0.0)
dt = Constant(1.0)

In [ ]:
bc = firedrake.DirichletBC(W.sub(0), u_0, inflow_ids)

In [ ]:
method = ...
params = {
    "bcs": bc,
    "form_compiler_parameters": {"quadrature_degree": 6},
    "solver_parameters": {"snes_linesearch_type": "nleqerr"},
}
stepper = irksome.TimeStepper(F, method, t, dt, w, **params)

In [ ]:
final_time = 400.0
num_steps = ...

for step in trange(num_steps):
    ...

## Analysis & visualization

First let's make a plot of the final thickness.

In [ ]:
u, M, h = w.subfunctions

fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.axis("off")
colors = firedrake.tripcolor(h, axes=ax)
fig.colorbar(colors, orientation="horizontal");

Many kinds of analyses that we want to do are expressible using the form language DSL.
For example, the code below calculates the total ice volume.

In [ ]:
volume = ...
print(f"Final ice volume: {volume / 1e9:0.0f} km³")

We can then plot the volume through time in order to assess whether the system has approached steady state.

In [ ]:
times = np.linspace(0.0, final_time, num_steps + 1)
volumes = ...

fig, ax = plt.subplots()
ax.set_xlabel("Time (years)")
ax.set_ylabel("Volume (km³)")
ax.plot(times, volumes);

Here's a more sophisticated example: plotting the fluxes of ice through each boundary segment.

In [ ]:
from firedrake import ds

u, M, h = w.subfunctions

ν = ...
influx = ...
outflux = ...

print(f"Influx:  {influx / 1e9:0.3f} km³/yr")
print(f"Outflux: {outflux / 1e9:0.3f} km³/yr")

Finally, the code below shows how to make a movie.

In [ ]:
%%capture

fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
colors = firedrake.tripcolor(
    hs[0], axes=ax, num_sample_points=1, shading="gouraud"
)
fig.colorbar(colors, orientation="horizontal")

In [ ]:
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=1)
def animate(h):
    colors.set_array(fn_plotter(h))

animation = FuncAnimation(fig, animate, tqdm(hs), interval=1e3/30)

In [ ]:
HTML(animation.to_html5_video())